# Data management: Conversion and integration of csv files on flights, airports and airlines into json files

In [ ]:
 # Installing libraries not present in google colab but necessaries for 
 # this script. You can skip this if you already have this libreries!
 !pip install --upgrade jsonschema
 !pip install --upgrade pymongo

In [ ]:
import pandas as pd  # for csv reader
import numpy as np  # for checking NaN values
import json
import jsonschema
from jsonschema import validate, Draft7Validator # pip install --upgrade jsonschema
import os
import time
# importing pymongo library
import pymongo
from pymongo import MongoClient

## MongoDB connection
Change these settings with your local machine settings. In my version I created 
a docker container with mongo db and a collection with 3 shards for the data.

In [ ]:
# Connecting to mongo client. Change arguments according to your machine 
client = MongoClient('mongos1', 27017)
# get the instance of BTS database
mongoDB = client.BTS

### Testing mongo database with some queries

In [ ]:
def query_execution():
  """
    query_execution() execute four different query on mongoDB and print result
    and time of execution!
  """

  
  # Query1: return the total average of all flights
  start_time = time.time()
  results = mongoDB.flight.aggregate([{"$unwind": "$flights"}, 
                                      {"$group": {"_id": "_id", 
                                                  "tot_avg_delay": {"$avg": "$flights.arr_delay"}}}, 
                                      {"$project": {"_id":0, "tot_avg_delay":1}}])
  print("Query1 result: " + results)
  print("Query1 seconds: %s " % (time.time() - start_time))

  # Query2: return the total average of all flights for the airline EV
  start_time = time.time()
  results = mongoDB.flight.aggregate([{"$match": {"flights.airline": "EV"}},
                                      {"$unwind": "$flights"}, 
                                      {"$match": {"flights.airline": "EV"}}, 
                                      {"$group": {"_id": "_id", 
                                                  "average": {"$avg": "$flights.arr_delay"}}}, 
                                      {"$project": {"_id":0, "average":1}}])
  print("Query2 result: " + results)
  print("Query2 seconds: %s " % (time.time() - start_time))

  # Query3: return the total average for the flight between SFO and LAX
  start_time = time.time()
  results = mongoDB.flight.aggregate([{"$match": {"origin": "SFO", "destination": "LAX"}},
                                      {"$unwind": "$flights"},  
                                      {"$group": {"_id": "_id", 
                                                  "average": {"$avg": "$flights.arr_delay"}}}, 
                                      {"$project": {"_id":0, "average":1}}])
  print("Query3 result: " + results)
  print("Query3 seconds: %s " % (time.time() - start_time))


  # Query4: return the total average of all flights ffor the year 2018
  start_time = time.time()
  results = mongoDB.flight.aggregate([{"$match": {"year": 2018}}, 
                                      {"$unwind": "$flights"}, 
                                      {"$group": {"_id": "_id",
                                                  "average": {"$avg": "$flights.arr_delay"}}}, 
                                      {"$project": {"_id":0, "average":1}}])
  print("Query4 result: " + results)
  print("Query4 seconds: %s " % (time.time() - start_time))



## Create validator for json files

In [ ]:
# load json schema for validate files
json_schema = json.loads(open('./Json-schema/JsonFlightSchema.json').read())

In [ ]:
def json_validator(json_schema, json_data):
  """
    validateJson() validates a json file against a proper schema


    :param json_schema: schema file for validating json data file
    :param json_data: file json to validate
    :return: True if file is valid against schema, False otherwise
  """ 

  # try to validate json data
  try:
    Draft7Validator(json_schema).validate(json_data)
  except jsonschema.exceptions.ValidationError as err:
    return False
  return True

## Data loading and preparation 

In [ ]:
def is_NaN(val):
  """
    is_NaN() check if a value is null for string and float types

    :param val: value to check
    :return: True if value is NaN or NaT or null, False otherwise
  """ 

  # check string NaN
  if isinstance(val, str) and not (val == '' or pd.isnull(val)):
    return False
  # check NaT/NaN value for date
  elif not pd.isnull(val):
    return False

  return True

#### Airlines data

In [ ]:
# get confidence with airline data
temp = pd.read_csv('./CSV-File/CSV-for-integration/airlines.csv')
temp.head()

In [ ]:
def create_airlines_dictionary():
  """
    create_airlines_dictionary() loads airline data and create a key-value 
    dictionary containing IATA-name pairs

    :return: Dictionary contains pairs with IATA codes and airline's name
  """ 

  # reading airline data 
  data_airlines = pd.read_csv('./CSV-File/CSV-for-integration/airlines.csv')

  airlines_dictionary = {}
  # for each line if IATA code is valid create a pair with airline's name
  # key = airline's IATA code, value = airline's name
  for line in data_airlines.itertuples(index=False): 
    if not is_NaN(line.IATA) and line.IATA != "-":
      airlines_dictionary[line.IATA.upper()] = line.Name
  
  return(airlines_dictionary)

#### Airports data

In [ ]:
# get confidence with airport data
temp = pd.read_csv('./CSV-File/CSV-for-integration/airports.csv')
temp.head()

In [ ]:
def create_airport_dictionary():
  """
    create_airport_dictionary() loads airport data and create a key-value 
    dictionary which contains key value pairs where:
    key: airport's iata code
    value: list of values as follow [icao code, name, airport type, 
                                     elevation_ft, latitude, longitude]

    :return: Dictionary contains pairs with IATA codes and a list of information
             for the airport
  """ 

  # reading airline data 
  data_airport = pd.read_csv('./CSV-File/CSV-for-integration/airports.csv')

  airport_dictionary = {}
  # for each line if IATA code is valid create a pair with airport's info
  # key = airport's IATA code
  # value: [icao, name, type, elevation_ft, latitude, longitude]
  for line in data_airport.itertuples(index=False): 
    if not is_NaN(line.iata_code):
      longi, lati = line.coordinates.split(",", 1)
      airport_dictionary[line.iata_code.upper()] = [line.gps_code, line.name,
                                               line.type, line.elevation_ft,
                                               float(lati), float(longi)]
  
  return(airport_dictionary)

#### Top 50 airports passengers data

In [ ]:
# get confidence with passengers data
col_names=['world_ranking', 'USA_ranking', 'country', 'city_state', 
           'IATA', 'tot_passengers', 'growth_percent'] 

temp = pd.read_excel('./CSV-File/CSV-for-integration/Passengers_2019.xlsx',header=2, names=col_names)
temp.head()

In [ ]:
def create_passengers_dictionary():
  """
    create_passengers_dictionary() loads airport passengers data and create a 
    key-value dictionary which contains key value pairs where:
    key: airport's iata code
    value: list of values as follow ["year;world_ranking;tot_passengers;growth_percent", ...]

    :return: Dictionary contains pairs with IATA codes and a list of passengers 
             data for the airport
  """ 

  # define column names for the dataFrame 
  col_names=['world_ranking', 'USA_ranking', 'country', 'city_state', 'IATA', 
            'tot_passengers', 'growth_percent'] 


  airport_passengers_dict = {}
  # read passengers data from file for 2018 and 2019
  for i in range(2018,2020):
    # reading excel data
    data_ap = pd.read_excel('./CSV-File/CSV-for-integration/Passengers_' + str(i) + '.xlsx',
                                            header=2, names=col_names)
    
    # For each line in the file, it adds the information to the dictionary 
    for line in data_ap.itertuples(index=False): 
      # corret growth info, because some file is in % and others not
      if line[6] < 0.1:
        growth = line[6]*100
      else:
        growth = line[6]

      # create string by concatenate information
      to_add = str(i) + ";" + str(line[0]) + ";" + str(line[5]) + ";" + str(growth)
      # get airport iata code
      airport_iata = line[4].upper()

      # check if the code already exist and append or create the values for the key
      if airport_iata in airport_passengers_dict.keys():
        (airport_passengers_dict[airport_iata]).append(to_add)
      else:
        airport_passengers_dict[airport_iata] = [to_add]
      

  return airport_passengers_dict

#### Flights data

In [ ]:
# get confidence with data
temp = pd.read_csv("./CSV-File/BTS-FlightData/2018_1.csv", usecols=list(range(0, 32)))
temp.head()

In [ ]:
def load_flight_csv(filepath):
  """
    load_flight_csv() Read the flight data files and prepare the dataset 
    for processing by converting the data fields to the right type 

    :param filepath: path from which to read the data 
    :return: Pandas dataframe with data
  """ 
  
  # read data from file
  flight_data = pd.read_csv(filepath, usecols=list(range(0, 32)))
  
  # I don't convert here time because there are some lines with NaN values.
  # NOTE: I save time as integer in json, so then I convert only line with valid values
  
  return flight_data

## Data management: Creating JSON files and uploading into mongoDB 
Each files groups year, month, origin airport, destination airport and day of the week <br>
(**NOTE:** year and month are grouped automatically as the files are already divided!) 

### Create json section about airports info

In [ ]:
# Init dictionaries for airlines, airports and passengers data
airline_iata_name_dic = create_airlines_dictionary()
airport_data_dic = create_airport_dictionary()
airport_passengers_dic = create_passengers_dictionary()

In [ ]:
def json_airport_info(year, airport_code, str_city_state, str_state_name):
  """
    json_airport_info() create a python dictionary with all the information 
    of a specific airport.

    :param year: Reference year to eventually enter the list with passengers data 
    :param airport_code: IATA code of the airport
    :param str_city_state: String consisting of "CityName, stateCode"
    :param str_state_name: String consisting of "StateName"  
    :return: Python dictionary with the followind data:
             {ICAO, name, type, elevation_ft, latitude, longitude, city_name,
             state_name, state_code}
  """

  # save airport iata code as upper string
  airport_iata = airport_code.upper()

  # get all info from airport dictionary, I'm sure the airport code exists in the 
  # dictionary as a key otherwise I'd have to use a try-catch or an if statement
  info_list = airport_data_dic[airport_iata]

  # info_list = [icao, name, airport type, elevation_ft, latitude, longitude]

  # split city name and state code
  city, state_code = str_city_state.split(",", 1)

  num_pass = []
  # Check if the we have info about passengers for this airport
  if airport_iata in airport_passengers_dic.keys():
    single_pass = {}
    
    # check if this airport have info of passengers for one or more year,
    # each year is saved as a string in a list into airport_passengers_dic
    # with key = iata_code of the airport
    # value = ["year;world_ranking;tot_passengers;growth_percent", ...]
    for str_val in airport_passengers_dic[airport_iata]:
      # split string of values
      actual_year, rank, tot_pass, growth = str_val.split(";",4)

      # If the year is <= the reference year I add it 
      # (i.e. If I am considering a 2018 flight I do not add info on 2019 passengers)
      if int(actual_year) <= int(year):
        single_pass["year"] = int(actual_year)
        single_pass["world_ranking"] = int(rank)
        single_pass["total_passengers"] = int(tot_pass)
        single_pass["growth_percent"] = float(growth)

    # append each year to the dictionary
    num_pass.append(single_pass)

  # create dictionary with airport info
  airport_info = {
      "ICAO": info_list[0],
      "name": info_list[1],
      "type": info_list[2],
      "elevation_ft": int(info_list[3]),
      "latitude": info_list[4],
      "longitude": info_list[5],
      "city_name": city,
      "state_name": str_state_name,
      "state_code": state_code.strip() # delete space
  }

  # If information exist I add the array with the info about passengers
  if num_pass != []:
    airport_info["num_passengers"] = num_pass

  return airport_info

### Create json section about flights

In [ ]:
def json_single_flight(fligth_data):
  """
    json_single_flight() create a python dictionary with all the information 
    of a specific flight. Only valid values will be added to the dictionary
    (i.e. NaN/null/NaT values will not be add, refers to db schema)

    :param fligth_data: flight data to parse
    :return: Python dictionary of the data
  """ 

  time_dic = {}

  time_dic['day_of_month'] = fligth_data.DAY_OF_MONTH

  # ailine's IATA code
  airline_IATA_code = fligth_data.OP_CARRIER.upper()
  time_dic['airline'] = airline_IATA_code
   
  # get the airline name from IATA codes by using the dictionary init before
  if airline_IATA_code in airline_iata_name_dic.keys():
    time_dic['airline_name'] = airline_iata_name_dic[airline_IATA_code]

  # departure time. I save time field as integer, in this way it's more
  # easy quering data from mongoDB
  time_dic['crs_dep_time'] = int(fligth_data.CRS_DEP_TIME)

  # saving departure time, cast will be always safe thanks to if
  if not is_NaN(fligth_data.DEP_TIME):
    time_dic['dep_time'] = int(fligth_data.DEP_TIME)

  # saving delay as integer, cast will be always safe thanks to if
  if not is_NaN(fligth_data.DEP_DELAY):
    time_dic['dep_delay'] = int(fligth_data.DEP_DELAY)

  # saving dep delay group as integer, cast will be always safe thanks to if
  if not is_NaN(fligth_data.DEP_DELAY_GROUP):
    time_dic['dep_delay_group'] = int(fligth_data.DEP_DELAY_GROUP)

  # arrival time (Does not contains NaN)
  time_dic['crs_arr_time'] = int(fligth_data.CRS_ARR_TIME)

  # saving arrival time, cast will be always safe thanks to if
  if not is_NaN(fligth_data.ARR_TIME):
    time_dic['arr_time'] = int(fligth_data.ARR_TIME)

  # saving delay as integer, cast will be always safe thanks to if
  if not is_NaN(fligth_data.ARR_DELAY):
    time_dic['arr_delay'] = int(fligth_data.ARR_DELAY)

  # saving arrival delay group as integer, cast will be always safe thanks to if
  if not is_NaN(fligth_data.ARR_DELAY_GROUP):
    time_dic['arr_delay_group'] = int(fligth_data.ARR_DELAY_GROUP)

  # check if flight was cancelled (No NaN)
  if fligth_data.CANCELLED == 1.0:
    time_dic['cancelled'] = True
    time_dic['cancellation_code'] = fligth_data.CANCELLATION_CODE
  else:
    time_dic['cancelled'] = False

  # check if flight was diverted
  if fligth_data.DIVERTED == 1.0:
    time_dic['diverted'] = True
  else:
    time_dic['diverted'] = False

  # saving crs Elapsed Time of Flight, cast will be always safe thanks to if
  if not is_NaN(fligth_data.CRS_ELAPSED_TIME):
    time_dic['crs_elapsed_time'] = int(fligth_data.CRS_ELAPSED_TIME)

  # saving real Elapsed Time of Flight, cast will be always safe thanks to if
  if not is_NaN(fligth_data.ACTUAL_ELAPSED_TIME):
    time_dic['actual_elapsed_time'] = int(fligth_data.ACTUAL_ELAPSED_TIME)

  # saving air Time of Flight, cast will be always safe thanks to if
  if not is_NaN(fligth_data.AIR_TIME):
    time_dic['air_time'] = int(fligth_data.AIR_TIME)

  # saving carrier_delay of Flight, cast will be always safe thanks to if
  if not is_NaN(fligth_data.CARRIER_DELAY):
    time_dic['carrier_delay'] = int(fligth_data.CARRIER_DELAY)

  # saving weather delay of Flight, cast will be always safe thanks to if
  if not is_NaN(fligth_data.WEATHER_DELAY):
    time_dic['weather_delay'] = int(fligth_data.WEATHER_DELAY)

  # saving nas delay of Flight, cast will be always safe thanks to if
  if not is_NaN(fligth_data.NAS_DELAY):
    time_dic['nas_delay'] = int(fligth_data.NAS_DELAY)

  # saving security delay of Flight, cast will be always safe thanks to if
  if not is_NaN(fligth_data.SECURITY_DELAY):
    time_dic['security_delay'] = int(fligth_data.SECURITY_DELAY)

  # saving late aircraft delay of Flight, cast will be always safe thanks to if
  if not is_NaN(fligth_data.LATE_AIRCRAFT_DELAY):
    time_dic['late_aircraft_delay'] = int(fligth_data.LATE_AIRCRAFT_DELAY)	

  return time_dic

### Create json basic structure for each .csv files

In [ ]:
def json_file_handler(json_dictionary, choice = 2):
  """
    json_file_handler() manages  the json dictionary based on user's choice.
    If choice == 1 then upload document on mongo db
    if choice is 2 then save the document into a specific path
    if choice == 3 then try to validate json file and return the result


    :param json_dictionary: Json dictionary to manage
    :param choice: user's choice for the function. Default choice is 2.

    :return: If choice == 1 return True if document is inserted, False otherwise
             If choice == 2 return None
             If choice == 3 return True if document is validate, False otherwise.
  """

  os.makedirs("./Json-schema/sample", exist_ok=True)   

  if choice == 1:
    # uploading document into mongoDB
    try:
        mongoDB.flight.insert_one(json_dictionary)
        return True
    except Exception as e:
        print("An exception occurred ::", e)
        return False
  elif choice == 2:
    # writing document appending time for unique name
    with open('./Json-schema/sample/sample' + str(time.time()) + '.json', 'w') as fp:
      json.dump(json_dictionary, fp)
    
    return True
  else:
    # writing temporary file for validation
    with open('./Json-schema/sample/temp.json', 'w') as fp:
      json.dump(json_dictionary, fp)
    f = open('./Json-schema/sample/temp.json',) 
    data = json.load(f) 

    # try to validate json file
    res = json_validator(json_schema, data)

    # Removing the temporary document
    os.remove("./Json-schema/sample/temp.json")
    
    return res
    

In [ ]:
def from_data_to_json(data_groups):
  """
    from_data_to_json() create a json dictionary with all the information 
    about flights starting from the grouped data. 
    NOTE: in this file data_groups are group by year, month, day_of_week, origin, destinarion

    :param data_groups: pandas groupby object that contains information 
                        about flight groups.
    :return: Nothing, directly upload files to mongoDB.
  """

  # Create a json file and upload it to mongo database for each group of 
  # day-origin-destination (Remember: each file contains a specific month and year)
  for group in data_groups:
    document = {}
    # list of all flights for a specific day, origin and destination
    flights = []
    for flight in group[1].itertuples():
      # Add basic information in case it is a new group
      if document == {}:
        document['year'] = flight.YEAR
        document['month'] = flight.MONTH
        document['day_of_week'] = flight.DAY_OF_WEEK
        document['origin'] = flight.ORIGIN.upper()
        document['origin_info'] = json_airport_info(flight.YEAR, flight.ORIGIN, 
                                                    flight.ORIGIN_CITY_NAME, 
                                                    flight.ORIGIN_STATE_NM)
        document['destination'] = flight.DEST.upper()
        document['destination_info'] = json_airport_info(flight.YEAR, flight.DEST, 
                                              flight.DEST_CITY_NAME, 
                                              flight.DEST_STATE_NM)
        document['distance'] = int(flight.DISTANCE)
        document['distance_group'] = flight.DISTANCE_GROUP
      
      # Append all flight information to the list
      flights.append(json_single_flight(flight))

    # add the list of all flight to the document
    document['flights'] = flights

    # Manage the json dictionary, 
    # 1 for upload into mongo, 2 for writing, 3 for test again schema
    # please note that 2 is computationally expensive because create
    # a milions of files!
    json_file_handler(document, 2)

In [ ]:
# elaborate all flight files in the directory
directory = "./CSV-File/BTS-FlightData"
for filename in os.listdir(directory):
  # printing feedback
  print("Executing: "  str(filename))

  # reading data on flights 
  flight_df = load_flight_csv(directory + "/" + filename)
  
  # group data, each file already grouped by year and month too!
  flight_df_groups = flight_df.groupby(['DAY_OF_WEEK', 'ORIGIN', 'DEST'])

  # convert data to json and upload to mongoDB
  from_data_to_json(flight_df_groups)
  
  # If you want to query the database step by step uncomment the following line
  # query_execution()

  # printing feedback
  print("DONE: " + str(filename))

In [ ]:
# If you have used the second json for the function json_file_handler
# and you want to download the zip file execute this line
!zip -r json_file.zip ./Json-schema/sample/ 

In [ ]:
# Try to execute some query on mongoDB and get result
query_execution()